In [1]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry
from api_key import APIKEY

API_KEY = APIKEY

with MPRester(API_KEY) as mpr:
    docs = mpr.materials.summary.search(
        energy_above_hull=(0, 0.008), band_gap=[0.85,2.15], is_stable=True, fields=["material_id"]
    )
    materials = [doc.material_id for doc in docs]

Retrieving SummaryDoc documents:   0%|          | 0/5081 [00:00<?, ?it/s]

In [2]:
def get_material_entries(material_ids, batch_size=6000):
    mp_entries = []
    with MPRester(API_KEY) as mpr:
        for i in range(0, len(material_ids), batch_size):
            batch_ids = material_ids[i:i + batch_size]
            material_entries = mpr.materials.search(material_ids=batch_ids)
            mp_entries.extend(material_entries)
    return mp_entries

mp_entries = get_material_entries(materials)

Retrieving MaterialsDoc documents:   0%|          | 0/5081 [00:00<?, ?it/s]

In [3]:
chemsys_list = [entry.chemsys for entry in mp_entries[:100] if hasattr(entry, 'chemsys')]
composition_list = [entry.composition for entry in mp_entries[:100] if hasattr(entry, 'composition')]
materials_gga = [material + "-GGA" for material in materials[:100]]
materials_id = materials[:100]

In [4]:
from pymatgen.core.composition import Composition

def remove_o_and_h(composition):
    new_composition_elements = []
    for element, count in composition.get_el_amt_dict().items():
        if element not in ['O', 'H']:
            new_composition_elements.append(f"{element}{int(count)}")
    new_composition_str = ' '.join(new_composition_elements)
    return Composition(new_composition_str)

filtered_molecules = [remove_o_and_h(molecule) for molecule in composition_list]

In [5]:
from pymatgen.core.composition import Composition

def calculate_percent_composition(molecule):
    total_atoms = sum(molecule.values())
    return {element.symbol: count / total_atoms for element, count in molecule.items()}

percent_compositions = [calculate_percent_composition(molecule) for molecule in filtered_molecules]

In [ ]:
import time
pbx_data = []
pbx_entries = []

with MPRester(API_KEY) as mpr:
    count = 0  # Initialize a counter variable
    for i, x in zip(chemsys_list, percent_compositions):
        count += 1  # Increment the counter at the start of each loop iteration
        try:
            pourbaix_entries = mpr.get_pourbaix_entries(i)
            pbx_entries.append(pourbaix_entries)
            pbx_diagram = PourbaixDiagram(entries=pourbaix_entries, comp_dict=x)
            pbx_data.append(pbx_diagram)
            time.sleep(10)
        except Exception as e:
            pbx_data.append('error')
        finally:
            print(f"Loop has run {count} times")  # Print the current loop count


In [7]:
matches = []

for index, comp in enumerate(composition_list):
    comp_formula = comp.formula
    found_match = False 

    try:
        for sublist in pbx_entries:
            if isinstance(sublist, list):
                for entry in sublist:
                    entry_str = str(entry)
                    if comp_formula and comp_formula in entry_str:
                        matches.append(entry)  
                        found_match = True
                        break

            if found_match:
                break

        if not found_match:
            print(f"No match found for: {comp_formula}")
            matches.append("None")

    except Exception as e:
        print(f"An error occurred at index {index} for composition {comp_formula}: {e}")
        matches.append("None")  


No match found for: Cs4 Np4 Cl8 O12
No match found for: As6 Xe12 F54


In [8]:
stabilities = []

for i, x in zip(pbx_data, matches):
    try:
        stability = i.get_decomposition_energy(x, 7, 0)
        formatted_stability = "{:.3g}".format(stability)
        stabilities.append(formatted_stability)
    except Exception as e:
        stabilities.append("error")

print(stabilities)


['2.5', '2.48', '0.211', '0.346', '2.39', '1.26', '0.403', '0.515', '1.59', '0.255', '0.26', '0.359', '0.39', '1.35', '1.24', '0.34', '0.316', '4.31', '3.83', '0.403', '2.27', '4.43', '0.583', '2.06', '0.529', '0.494', '0.161', '2.04', '0.457', '0.0506', '0.0902', '0.782', '2.37', '0.989', '0.402', '0.443', '0.788', '0.977', '2.25', '0.991', '1.09', '0.588', '0.381', '0.531', '1.37', '1.26', '0.949', '0.818', '0.822', '0.921', '0.937', '0.601', '0.281', '1.6', '0.764', 'error', '1.04', '0.0722', '1.26', '0.0679', '0.586', '0.582', '1.06', '1.05', '0.38', '0.7', '1.22', 'error', '1.67', '0.263', '0.799', '0.427', '0.558', '0.488', '0.952', '0.227', '0.381', '0.819', '1.09', '0.865', '0.267', '0.274', '3.74', '0.19', '0.642', '0.152', '0.585', '0.595', '1.24', '0.608', '1.21', '0.00725', '0.674', '0.54', '0.996', '1.49', '1.16', '0.831', '0.266', '0.214']


In [9]:
combined_list = list(zip(materials_id, stabilities))
data_without_error = [item for item in combined_list if item[1] != 'error']
filtered_data = [item for item in data_without_error if float(item[1]) <= 0.5]
mp_identifiers = [item[0] for item in filtered_data]

In [10]:
print("total materials:",len(materials_id))
print("materials without error:",len(data_without_error))
print("stable materials:",len(filtered_data))


total materials: 100
materials without error: 98
stable materials: 34
